In [203]:
import pandas as pd
import altair as alt
import numpy as np

data = pd.read_csv("app-lab-data.csv") # This data is for two semesters (Fall 2021/ Spring 2022)

# Sun 3/31/2024 - the date that room reservations went live!

data.head()

,id,name,date,timeIn,timeOut,Duration,reason,classification,staff,comments,PID,hasPID,firstTime,heard_about_al_through
0,371,Joyce Zhang,9/3/21,13:35:00,13:47:00,0:12:00,Web-based survey,project/workshop/staff,Lasya,"Issue with heroku deployment, had a 404 error ...",0,False,False,NaN
1,390,Joyce Zhang,9/9/21,12:41:00,18:14:00,5:33:00,work,work/study/hang,NaN,set up her mac computer to do dev work on a mo...,0,False,False,NaN
2,399,Joyce Zhang,9/10/21,14:57:00,16:23:00,1:26:00,work,work/study/hang,Lia,She had a problem accessing a key for a backen...,0,False,False,NaN
3,510,Joyce Zhang,10/7/21,11:41:00,12:47:00,1:06:00,work,work/study/hang,NaN,NaN,0,False,False,NaN
4,574,Xinyi Zhang,10/15/21,10:58:00,12:00:00,1:02:00,work,work/study/hang,NaN,NaN,1,False,False,NaN


In [204]:
# Create convert duration to number of days per reservation
data['Duration (days)'] = pd.to_timedelta(data['Duration']).dt.total_seconds() / (24 * 60 * 60) 

# Create table for top 10 students in duration of time.  
duration_by_pid = data.groupby(['PID'])
duration_by_pid = duration_by_pid['Duration (days)'].sum().reset_index()
duration_by_pid = duration_by_pid.sort_values(by='Duration (days)', ascending=False)
duration_by_pid = duration_by_pid.head(10)

duration_by_pid

,PID,Duration (days)
55,730325023,4.511111
92,730399319,3.809722
50,730320402,3.472222
139,730515295,3.318750
47,730318278,2.325000
102,730409430,2.247917
65,730355643,2.143750
71,730367015,1.877083
141,730520865,1.611111
70,730366571,1.558333


In [205]:
# Visualization of the top users in each space.

chart = alt.Chart(duration_by_pid).mark_bar().encode(
    y=alt.Y('PID:O', title='User ID', sort="-x"),
    x=alt.X('Duration (days)', title='Duration (Days)'),
    tooltip=alt.Tooltip('Duration (days)')
).properties(
    title='Total Time per User (Days)'
)

chart


alt.Chart(...)

In [206]:
# Do the leaderboard per day. 

# Convert string to TimeDelta Object.
data['date'] = pd.to_datetime(data['date'], format="%m/%d/%y")

data['timeIn'] = pd.to_timedelta(data['timeIn'])

data['start'] = data['date'] + data['timeIn']

data['day_of_week'] = data['start'].dt.day_name()

data['civilian_time'] = data['start'].dt.strftime("%I %p").str.lstrip("0")

reservations_per_hour = data.groupby(["day_of_week", "civilian_time"]).size().reset_index(name="count")

reservations_per_hour.head(5)


,day_of_week,civilian_time,count
0,Friday,1 PM,11
1,Friday,10 AM,1
2,Friday,11 AM,2
3,Friday,12 PM,13
4,Friday,2 PM,12


In [207]:
# Filter out Saturday and Sunday
reservations_per_hour_weekdays = reservations_per_hour[reservations_per_hour['day_of_week'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

chart = alt.Chart(reservations_per_hour_weekdays).mark_bar().encode(
    x=alt.X('civilian_time:N', title='Hour of the Day', sort=['10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM']),
    y=alt.Y('count:Q', title='Reservations'),
    column=alt.Column('day_of_week:N', title=None, sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
).properties(
    height=150,
    title='Popular Times in the App Lab'
)

chart.show()

alt.Chart(...)